In [1]:
import os

dataPath = '/home/endre/Endre_finance_ml_data/'
splittedPath = dataPath + 'generated/WIKI_PRICES_SPLIT/'

numberOfFiles = 0
namePathDict = {}
for subdir, dirs, files in os.walk(splittedPath):
    for file in files:
        filePath = os.path.join(subdir, file)
        namePathDict[file.split('.')[0]] = filePath
        numberOfFiles += 1

print("Total number of files:", numberOfFiles)

Total number of files: 3188


In [2]:
import pandas as pd
# import numpy as np

def readTickerAsPandas(ticker):
    return pd.read_csv(splittedPath+ticker+'.csv')

In [3]:
class BadNumberInArrayError(ValueError):
    '''There was a bad number in the array'''

def percentChange(old, new):
    return ((new-old)/old)*100.0

import math
def checkBadNumbers(rangeName, array):
    for num in array:
        if (math.isnan(num) or not math.isfinite(num)):
            raise BadNumberInArrayError("Got a bad number in this array: {} -> {}".format(rangeName, array))

def featurizeTicker(rangeNames, features, labels, ticker, df):
    selected = df[["date", "adj_close", "adj_volume"]]
    
    # 21 days per month, 12 months = roughly a year.
    numDaysPeriod = 21
    numPeriods = 18
    jumpDays = 3  # Number of days to jump between each feature (reduce number of feature sets)

    rangeDays = numDaysPeriod * numPeriods

    # RANGE: "the entire year" - which spans the feature set
    # PERIOD: "each month"
    
    featureSetDate = None
    
    idx = 0
    for featureSetIdx in range(rangeDays-1, selected.shape[0] - numDaysPeriod, jumpDays):
        # DEBUG::
        idx += 1
        if (idx > rangeDays*43525324532):
            break
        # DEBUG END!

        # The "name" of this feature set
        featureSetDate = selected.iloc[featureSetIdx, 0]

        # The start of this feature set's range
        rangeStartIdx = featureSetIdx-rangeDays+1
        # The end of this feature set's range
        rangeEndIdx = featureSetIdx
        # The start of this feature set's range
        rangeStartDate = selected.iloc[rangeStartIdx, 0]
        
        # The volume average over the range preceeding this feature date
        averageVolumeOverRange = selected.iloc[rangeStartIdx:rangeEndIdx+1, 2].mean()

        rangeStartPrice = selected.iloc[rangeStartIdx, 1]

        ## NAME THIS RANGE
        rangeName = []
        rangeName.append(ticker)
        rangeName.append(featureSetDate)
        
        ## CREATE THE FEATURES INTO 'features'
        rangeFeatures = []
        for preceedingPeriodIdx in range(-numPeriods+1, 1):
            periodStartIdx = featureSetIdx + (preceedingPeriodIdx - 1) * numDaysPeriod + 1
            periodEndIdx = featureSetIdx + preceedingPeriodIdx * numDaysPeriod
            periodStartDate = selected.iloc[periodStartIdx, 0]
            periodEndDate = selected.iloc[periodEndIdx, 0]

            periodStartPrice = selected.iloc[periodStartIdx, 1]
            periodEndPrice = selected.iloc[periodEndIdx, 1]

            f_periodPriceChange = percentChange(periodStartPrice, periodEndPrice)
            f_fromStartRangeToPeriodEndPriceChange = percentChange(rangeStartPrice, periodEndPrice)

            averageVolumeOverPeriod = selected.iloc[periodStartIdx:periodEndIdx+1, 2].mean()
            f_averageVolumeRatio = averageVolumeOverPeriod / averageVolumeOverRange

            # print("   #{}: {} - {}, f_periodPriceChange:{}, f_fromStartRangeToPeriodEndPriceChange:{}, f_averageVolumeRatio:{}".\
            #      format(preceedingPeriodIdx, periodStartDate, periodEndDate, f_periodPriceChange, f_fromStartRangeToPeriodEndPriceChange, f_averageVolumeRatio))

            rangeFeatures.append(f_periodPriceChange)
            rangeFeatures.append(f_fromStartRangeToPeriodEndPriceChange)
            rangeFeatures.append(f_averageVolumeRatio)
        
        checkBadNumbers(rangeName, rangeFeatures)
        
        ## CREATE THE LABELS INTO 'labels'
        rangeLabels = []
        featureSetEndPrice = selected.iloc[featureSetIdx, 1]
        rangeLabels.append(percentChange(featureSetEndPrice, selected.iloc[featureSetIdx+5, 1]))
        rangeLabels.append(percentChange(featureSetEndPrice, selected.iloc[featureSetIdx+10, 1]))
        rangeLabels.append(percentChange(featureSetEndPrice, selected.iloc[featureSetIdx+15, 1]))
        rangeLabels.append(percentChange(featureSetEndPrice, selected.iloc[featureSetIdx+20, 1]))
        
        checkBadNumbers(rangeName, rangeLabels)

        # ----- Now all features and labes should be checked, so we can safely add it to the lists
        rangeNames.append(rangeName)
        features.append(rangeFeatures)
        labels.append(rangeLabels)
        
    # Return last feature set date
    return featureSetDate

In [4]:
from timeit import default_timer as timer

rangeNames = []
features = []
labels = []

total_start = timer()
idx_ticker = 0
total_tickers = len(namePathDict)
for ticker, path in namePathDict.items():
    idx_ticker += 1

    # DEBUG::
    if (idx_ticker > 100000):
        print ("EARLY BREAK!")
        break
    # DEBUG END!
    
    start = timer()
    df = readTickerAsPandas(ticker)
    try:
        lastFeatureSetDate = featurizeTicker(rangeNames, features, labels, ticker, df)
        millis = (timer()-start) * 1000
        print ("#{}/{}: {}: {}, last:{} :: took {} ms - current length: {}".\
               format(idx_ticker, total_tickers, ticker, df.shape, lastFeatureSetDate, millis, len(features)))
    except (ValueError, ZeroDivisionError) as err:
        print ("## Got error for Ticker '{}' - skipping rest of it.. \n  {}\n".format(ticker, err))

print("Total time: {}".format((timer() - total_start)*1000))


#1/3188: QTM: (4499, 14), last:2017-05-15 :: took 23890.460891998373 ms - current length: 1367
#2/3188: HTGC: (3027, 14), last:2017-05-17 :: took 15287.54881300847 ms - current length: 2244
#3/3188: ED: (11974, 14), last:2017-05-16 :: took 67251.1877809884 ms - current length: 6103
#4/3188: FIVN: (807, 14), last:2017-05-17 :: took 2379.3421770096757 ms - current length: 6240
#5/3188: CVEO: (778, 14), last:2017-05-16 :: took 2200.251449015923 ms - current length: 6367
#6/3188: NCI: (5210, 14), last:2017-05-15 :: took 28090.391563018784 ms - current length: 7971
#7/3188: LDOS: (2685, 14), last:2017-05-17 :: took 13455.646590009565 ms - current length: 8734
#8/3188: IGTE: (4665, 14), last:2015-06-02 :: took 24844.920207979158 ms - current length: 10157
#9/3188: AAPL: (9207, 14), last:2017-05-17 :: took 51244.01388000115 ms - current length: 13094
#10/3188: HEOP: (4515, 14), last:2017-03-02 :: took 24079.898925992893 ms - current length: 14467
#11/3188: BRKR: (4243, 14), last:2017-05-16 ::

#86/3188: BIOL: (6194, 14), last:2017-05-15 :: took 33893.830717977835 ms - current length: 117480
#87/3188: MBII: (976, 14), last:2017-05-16 :: took 3366.839709022315 ms - current length: 117673
#88/3188: ESCA: (6377, 14), last:2017-05-15 :: took 34884.41370698274 ms - current length: 119666
#89/3188: CSRA: (390, 14), last:None :: took 4.9195569881703705 ms - current length: 119666
#90/3188: KRC: (5129, 14), last:2017-05-15 :: took 27652.316761988914 ms - current length: 121243
#91/3188: VECO: (5678, 14), last:2017-05-15 :: took 30676.44952001865 ms - current length: 123003
#92/3188: MATX: (11132, 14), last:2017-05-15 :: took 62839.03127600206 ms - current length: 126581
#93/3188: TRXC: (6194, 14), last:2017-05-15 :: took 34089.27351501188 ms - current length: 128513
#94/3188: GLOG: (1312, 14), last:2017-05-16 :: took 5329.112462000921 ms - current length: 128818
#95/3188: NMRX: (5864, 14), last:2017-05-15 :: took 31850.896338000894 ms - current length: 130640
#96/3188: FRGI: (1294, 1

#170/3188: Z: (463, 14), last:2017-05-16 :: took 393.22624399210326 ms - current length: 234407
#171/3188: MIL: (4976, 14), last:2016-01-11 :: took 26657.39298099652 ms - current length: 235933
#172/3188: CI: (8881, 14), last:2017-05-16 :: took 49426.07961400063 ms - current length: 238761
#173/3188: ANIP: (4306, 14), last:2017-05-16 :: took 23105.716496997047 ms - current length: 240064
#174/3188: NBCB: (414, 14), last:2014-12-01 :: took 113.01503697177395 ms - current length: 240070
#175/3188: MTSC: (6862, 14), last:2017-05-16 :: took 37782.99039599369 ms - current length: 242225
#176/3188: AVB: (5859, 14), last:2017-05-17 :: took 32037.983383983374 ms - current length: 244046
#177/3188: RTRX: (1159, 14), last:2017-05-16 :: took 4451.1760890018195 ms - current length: 244300
#178/3188: IGT: (6862, 14), last:2017-05-16 :: took 37786.75387401017 ms - current length: 246455
#179/3188: SCBT: (4380, 14), last:2014-05-28 :: took 23260.27264801087 ms - current length: 247783
#180/3188: CIX:

#254/3188: UAM: (6827, 14), last:2017-03-24 :: took 37625.33094495302 ms - current length: 346546
#255/3188: PTIE: (4258, 14), last:2017-05-16 :: took 22606.573417957406 ms - current length: 347833
#256/3188: CCXI: (1348, 14), last:2017-05-16 :: took 5604.942601989023 ms - current length: 348150
#257/3188: DELL: (6388, 14), last:2014-11-12 :: took 35071.2708819774 ms - current length: 350147
#258/3188: ANGI: (1403, 14), last:2017-05-15 :: took 5863.756015023682 ms - current length: 350482
#259/3188: RCL: (6080, 14), last:2017-05-15 :: took 33253.321937983856 ms - current length: 352376
#260/3188: MEI: (6863, 14), last:2017-05-15 :: took 37736.35808401741 ms - current length: 354531
#261/3188: MDRX: (4504, 14), last:2017-05-16 :: took 24006.465065001976 ms - current length: 355900
#262/3188: RYN: (5874, 14), last:2017-05-17 :: took 32099.908317963127 ms - current length: 357726
#263/3188: CSS: (6362, 14), last:2017-05-15 :: took 34852.407007012516 ms - current length: 359714
#264/3188: 

#337/3188: TYC: (7295, 14), last:2016-08-02 :: took 40462.15808700072 ms - current length: 458064
#338/3188: ASEI: (8070, 14), last:2016-08-10 :: took 45959.071863035206 ms - current length: 460622
#339/3188: CMCSK: (6482, 14), last:2015-11-09 :: took 35611.384253017604 ms - current length: 462650
#340/3188: LMNX: (4326, 14), last:2017-05-17 :: took 22918.561102997046 ms - current length: 463960
#341/3188: DFZ: (5661, 14), last:2014-08-05 :: took 30877.13919702219 ms - current length: 465715
#342/3188: SBH: (2662, 14), last:2017-05-16 :: took 13210.859593993519 ms - current length: 466470
#343/3188: OSIS: (4959, 14), last:2017-05-17 :: took 26659.531180979684 ms - current length: 467991
#344/3188: SRI: (4953, 14), last:2017-05-17 :: took 26759.455236024223 ms - current length: 469510
#345/3188: ARNA: (4248, 14), last:2017-05-17 :: took 22646.733761997893 ms - current length: 470794
#346/3188: TMHC: (1056, 14), last:2017-05-17 :: took 3894.4265859900042 ms - current length: 471014
#347/

...

#2804/3188: NEON: (6372, 14), last:2017-05-17 :: took 34594.70793302171 ms - current length: 3979571
#2805/3188: CBZ: (5575, 14), last:2017-05-16 :: took 30217.238245008048 ms - current length: 3981297
#2806/3188: CSL: (7464, 14), last:2017-05-17 :: took 40955.02823998686 ms - current length: 3983653
#2807/3188: IPXL: (5411, 14), last:2017-05-15 :: took 29395.564225967973 ms - current length: 3985324
#2808/3188: LH: (6859, 14), last:2017-05-16 :: took 37691.24840700533 ms - current length: 3987478
#2809/3188: IVZ: (5491, 14), last:2017-05-16 :: took 29744.31503401138 ms - current length: 3989176
#2810/3188: BKS: (5974, 14), last:2017-05-16 :: took 32521.552120975684 ms - current length: 3991035
#2811/3188: PZG: (2798, 14), last:2017-05-15 :: took 14065.02800801536 ms - current length: 3991835
#2812/3188: BEE: (2854, 14), last:2015-11-10 :: took 14290.08689202601 ms - current length: 3992654
#2813/3188: CBS: (2903, 14), last:2017-05-15 :: took 14594.962817966007 ms - current length: 399

#2886/3188: USAP: (5658, 14), last:2017-05-17 :: took 30670.434257015586 ms - current length: 4099333
#2887/3188: TXN: (11362, 14), last:2017-05-16 :: took 63538.935907010455 ms - current length: 4102988
#2888/3188: IDRA: (5386, 14), last:2017-05-16 :: took 28938.166014035232 ms - current length: 4104651
#2889/3188: ORM: (999, 14), last:2017-05-17 :: took 3532.4096370022744 ms - current length: 4104852
#2890/3188: IL: (1625, 14), last:2016-12-14 :: took 7154.367308015935 ms - current length: 4105261
#2891/3188: SEE: (7427, 14), last:2017-05-15 :: took 40895.301638985984 ms - current length: 4107604
#2892/3188: UNIS: (1808, 14), last:2017-03-17 :: took 8176.651895977557 ms - current length: 4108074
#2893/3188: AZO: (6605, 14), last:2017-05-15 :: took 36115.97450799309 ms - current length: 4110143
#2894/3188: ROG: (9396, 14), last:2017-05-17 :: took 52187.71991803078 ms - current length: 4113143
#2895/3188: CBI: (5090, 14), last:2017-05-15 :: took 27213.67733104853 ms - current length: 4

#2968/3188: HY: (1185, 14), last:2017-05-17 :: took 4563.282532966696 ms - current length: 4210658
#2969/3188: PTGI: (1197, 14), last:2014-03-13 :: took 4627.8661449905485 ms - current length: 4210925
#2970/3188: BOFI: (3086, 14), last:2017-05-15 :: took 15626.701358007267 ms - current length: 4211821
#2971/3188: WNC: (6450, 14), last:2017-05-17 :: took 35102.24695899524 ms - current length: 4213839
#2972/3188: NFX: (5941, 14), last:2017-05-16 :: took 32032.134062959813 ms - current length: 4215687
#2973/3188: HBI: (2714, 14), last:2017-05-15 :: took 13457.246790989302 ms - current length: 4216459
#2974/3188: BHE: (6796, 14), last:2017-05-16 :: took 37124.87555498956 ms - current length: 4218592
#2975/3188: CUNB: (2919, 14), last:2017-05-17 :: took 14642.727886035573 ms - current length: 4219433
#2976/3188: NYLD: (988, 14), last:2017-05-16 :: took 3423.030900012236 ms - current length: 4219630
#2977/3188: LVLT: (4836, 14), last:2017-05-17 :: took 25739.330950018484 ms - current length:

#3050/3188: PNR: (11132, 14), last:2017-05-15 :: took 62316.803781024646 ms - current length: 4338746
#3051/3188: CPRT: (5855, 14), last:2017-05-15 :: took 31564.886796986684 ms - current length: 4340565
#3052/3188: TE: (7985, 14), last:2016-05-27 :: took 44051.25229997793 ms - current length: 4343094
#3053/3188: UBSI: (6862, 14), last:2017-05-16 :: took 37488.632094988134 ms - current length: 4345249
#3054/3188: LYB: (1798, 14), last:2017-05-16 :: took 8097.574261017144 ms - current length: 4345716
#3055/3188: HW: (5463, 14), last:2017-04-05 :: took 29426.17341602454 ms - current length: 4347405
#3056/3188: CLX: (8635, 14), last:2017-05-16 :: took 47759.80845297454 ms - current length: 4350151
#3057/3188: CPB: (8058, 14), last:2017-05-17 :: took 44413.31238002749 ms - current length: 4352705
#3058/3188: TWMC: (6129, 14), last:2017-05-17 :: took 33253.65992198931 ms - current length: 4354616
#3059/3188: ELLI: (1553, 14), last:2017-05-15 :: took 6692.33400200028 ms - current length: 435

#3132/3188: ATLO: (4362, 14), last:2017-05-17 :: took 23151.470140961464 ms - current length: 4459327
#3133/3188: GCO: (8058, 14), last:2017-05-17 :: took 44496.35647499235 ms - current length: 4461881
#3134/3188: CVD: (4572, 14), last:2015-01-16 :: took 24340.164924040437 ms - current length: 4463273
#3135/3188: AIZ: (3365, 14), last:2017-05-15 :: took 17176.80484900484 ms - current length: 4464262
#3136/3188: MXWL: (6862, 14), last:2017-05-16 :: took 37767.1303889947 ms - current length: 4466417
#3137/3188: WTFC: (4825, 14), last:2017-05-16 :: took 25850.70309601724 ms - current length: 4467893
#3138/3188: NFLX: (3794, 14), last:2017-05-15 :: took 19752.295155019965 ms - current length: 4469025
#3139/3188: MITK: (5554, 14), last:2017-05-16 :: took 29945.220998022705 ms - current length: 4470744
#3140/3188: LTXC: (6086, 14), last:2014-04-15 :: took 33208.2414960023 ms - current length: 4472640
#3141/3188: VRA: (1675, 14), last:2017-05-16 :: took 7428.095415001735 ms - current length: 

In [5]:
print("Total time: {}".format((timer() - total_start)*1000))
print("RangeNames:{}, Features:{}, Labels:{}".format(len(rangeNames), len(features), len(labels)))

Total time: 79260087.05409901
RangeNames:4541600, Features:4541600, Labels:4541600


In [6]:
import pickle

result = {'rangeNames': rangeNames, 'features': features, 'labels': labels}
pickle.dump(result, open("RangeNamesFeaturesAndLabels.pickle", "wb"))

In [7]:
idx_ticker

3188